# Create Heat Map for significant Cell Cycle genes

This notebook looks at the significant genes in at least one cancer in the DNA Replication pathway. Pancancer heat maps are created with circle size showing significance and color showing differences in median.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

import plot_utils as p

# Step 1: Run GSEA for significant genes in at least 1 cancer

First read in sig_pval_all_proteins.csv into a df. This csv file contains only genes with a significant p-value in at least one cancer. Then run GSEA using a list of genes from the df.

In [2]:
root = R'~\Github\WhenMutationsDontMatter\PTEN\Step_3_trans_effect\csv'
sig_df = pd.read_csv(root+R'\sig_pval_heatmap.csv')

prot_list = list(sig_df.Proteomics) # list of genes with a sig pval in >= 1 cancer
prot_enr = gp.enrichr(gene_list = prot_list, description='Tumor_partition', gene_sets='Reactome_2016', 
                       outdir='/Enrichr')

In [3]:
prot_enr.res2d.iloc[[9]]

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
9,Transport of Mature Transcript to Cytoplasm Ho...,46/74,1.962264e-22,3.002265e-20,0,0,4.644166,232.128295,NUP205;NUP107;SEH1L;NUP188;EIF4A3;NUP160;NXF1;...,Reactome_2016


# Step 2: Get the list of significant genes 

In [4]:
index = 9
trans = prot_enr.res2d.Genes[index]
genes = trans.split(';')
print(prot_enr.res2d.Term[index])
print('total genes:',len(genes))

Transport of Mature Transcript to Cytoplasm Homo sapiens R-HSA-72202
total genes: 46


# Step 3: Create HeatMap

Slice out genes from the DNA Replication pathway from the df with genes sig in > 1 cancer.

In [26]:
# sig > 1 cancer
bool_df = sig_df.Proteomics.isin(genes)
plot_df = sig_df[bool_df]
len(plot_df.Proteomics.unique())

46

In [27]:
# Only include p-values < a certain cutoff
a = 0.05
plot_df = plot_df.loc[plot_df['P_Value'] <= a]

In [38]:
p.plotCircleHeatMap(plot_df, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=200, plot_width= 1000, legend_min = 5e-3, legend_max = 0.05)

Loading BokehJS ...

# Step 4: Create a HeatMap with both +/- differences in median

 Read in the pos_neg_df.csv to create a df with only genes that have a pos and neg difference in median in different cancers. Slice out the genes that have a pos and neg difference in median in the pathway using the list of genes with a significant p-value in the pathway. 

In [30]:
pos_neg_df = pd.read_csv(root+R'\pos_neg_df.csv')

In [31]:
get = pos_neg_df.Proteomics.isin(genes) # bool df where True has both pos and neg
genes_pn = pos_neg_df[get] # Keep only genes with pos and neg
genes_pn.Proteomics.unique()

array(['NUP37', 'NUP153', 'DDX39A', 'NUP210'], dtype=object)

In [32]:
# Only include p-values < a certain cutoff
genes_pn = genes_pn.loc[genes_pn['P_Value'] <= a]

In [37]:
p.plotCircleHeatMap(genes_pn, circle_var = 'P_Value', color_var ='Medians', x_axis = 'Proteomics', y_axis = 'Cancer',
                    plot_height=200, plot_width=300, legend_min = 5e-3, legend_max = 0.05)

Loading BokehJS ...

# Sig in multiple cancers

In [39]:
mult_sig_df = pd.read_csv(root+R'\mult_sig_pval_heatmap.csv')

mult_sig_list = list(mult_sig_df.Proteomics.unique()) # list of genes with a sig pval in > 1 cancer
enr2 = gp.enrichr(gene_list = mult_sig_list, description='Tumor_partition', gene_sets='Reactome_2016', 
                       outdir='/Enrichr')

In [13]:
enr2.res2d.head(3)

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,Gene Expression Homo sapiens R-HSA-74160,82/1631,4.851338e-19,7.422548e-16,0,0,2.906122,122.550749,DDX47;WDR3;WDR4;PTEN;HNRNPU;NAT10;ZC3H8;ADAR;P...,Reactome_2016
1,Processing of Capped Intron-Containing Pre-mRN...,28/193,4.899234e-18,3.747914e-15,0,0,8.385995,334.244415,DDX5;DHX9;SF3B6;HNRNPU;PRPF19;USP39;ELAVL1;EFT...,Reactome_2016
2,"Cell Cycle, Mitotic Homo sapiens R-HSA-69278",38/462,1.553184e-15,7.921238e-13,0,0,4.754398,162.117689,TOP2A;AHCTF1;PCNA;MCM7;DCTN1;BUB1B;PPP2R2A;ORC...,Reactome_2016


In [14]:
index2 = 2
trans2 = enr2.res2d.Genes[index2]
genes_mult = trans2.split(';')
print(enr2.res2d.Term[index2])
print('total genes:',len(genes_mult))

Cell Cycle, Mitotic Homo sapiens R-HSA-69278
total genes: 38


In [15]:
bool_df = mult_sig_df.Proteomics.isin(genes_mult)
plot_df2 = mult_sig_df[bool_df]
plot_df2.Proteomics.unique()

array(['DCTN1', 'ACTR1A', 'MAD2L1', 'MCM4', 'PCNA', 'MCM5', 'MCM2',
       'PCM1', 'MCM7', 'TPR', 'MCM3', 'NUP153', 'RFC3', 'CDK11B', 'XPO1',
       'PRKCB', 'RFC2', 'NUF2', 'RAB8A', 'RFC4', 'RANBP2', 'CENPF',
       'NUP155', 'GINS4', 'NDC80', 'OPTN', 'ORC5', 'PPP2R2A', 'PPP2R2D',
       'TPX2', 'NUP85', 'RAE1', 'GINS2', 'BUB1B', 'TOP2A', 'NUP133',
       'AHCTF1', 'NUP210'], dtype=object)

In [16]:
# Only include p-values < a certain cutoff
a = 0.05
plot_df2 = plot_df2.loc[plot_df2['P_Value'] <= a]

In [45]:
p.plotCircleHeatMap(plot_df2, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=250, plot_width=900, legend_min = 5e-3, legend_max = 0.05)

Loading BokehJS ...

# DNA Initiation and Synthesis

In [81]:
narrowed_cell_cycle = ['DCTN1', 'ACTR1A', 'MAD2L1', 'MCM4', 'PCNA', 'MCM5', 'MCM2','PCM1',
              'MCM7', 'TPR', 'MCM3', 'NUP153', 'RFC3', 'CDK11B', 'XPO1','PRKCB', 'RFC2', 'NUF2', 
              'RAB8A', 'RFC4', 'RANBP2', 'CENPF', 'GINS4', 'NDC80', 'OPTN', 'PPP2R2A', 'PPP2R2D',
              'TPX2', 'GINS2', 'BUB1B', 'TOP2A','NUP210']

dna_synth = ['MCM4', 'PCNA', 'MCM5', 'MCM2','PCM1', 'MCM7', 'MCM3', 'RFC3', 
             'RFC2', 'RFC4', 'GINS4', 'GINS2', 'TOP2A']

dynactin = ['DCTN1','ACTR1A']
spindle = ['BUB1B', 'CENPF', 'MAD2L1', 'NDC80', 'NUF2', 'PCM1', 'TPX2','DCTN1','ACTR1A']

checkpoints = ['BUB1B','PPP2R2A', 'PPP2R2D', 'TOPBP1','MAD2L1']

np_transport = ['TPR','NUP153','NUP210', 'XPO1','RANBP2']
other_transport = ['RAB8A']
other = ['PRKCB','OPTN','CDK11B']

path_list = {'DNA_Synth':dna_synth, 'Spindle':spindle, 'Dynactin': dynactin, 'Checkpoints':checkpoints,
            'Nuclear_Pore_Transport':np_transport}


In [82]:
dfs = {}
for path in path_list:
    bool_df = mult_sig_df.Proteomics.isin(path_list[path])
    df = mult_sig_df[bool_df]
    df = df.loc[df['P_Value'] <= 0.05]
    dfs[path] = df

In [83]:
for path in dfs:
    print(path)
    p.plotCircleHeatMap(dfs[path], circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=250, plot_width=500, legend_min = 5e-3, legend_max = 0.05)

DNA_Synth


Loading BokehJS ...

Spindle


Loading BokehJS ...

Dynactin


Loading BokehJS ...

Checkpoints


Loading BokehJS ...

Nuclear_Pore_Transport


Loading BokehJS ...

In [84]:
bool_df = mult_sig_df.Proteomics.isin(narrowed_cell_cycle)
df = mult_sig_df[bool_df]
df = df.loc[df['P_Value'] <= 0.05]

In [86]:
p.plotCircleHeatMap(df, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=250, plot_width=700, legend_min = 5e-3, legend_max = 0.05)

Loading BokehJS ...

In [18]:
get = pos_neg_df.Proteomics.isin(genes_mult) # bool df where True has both pos and neg
genes_pn_mult = pos_neg_df[get] # Keep only genes with pos and neg
genes_pn_mult.Proteomics.unique()

array(['MCM4', 'MCM5', 'MCM2', 'PCM1', 'MCM7', 'MCM3', 'NUP153', 'CDK11B',
       'RFC2', 'NUF2', 'RFC4', 'CENPF', 'GINS4', 'NDC80', 'PPP2R2A',
       'TPX2', 'GINS2', 'BUB1B', 'TOP2A', 'NUP210'], dtype=object)

In [46]:
# Only include p-values < a certain cutoff
genes_pn_mult = genes_pn_mult.loc[genes_pn_mult['P_Value'] <= a]

In [47]:
p.plotCircleHeatMap(genes_pn_mult, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width=600)

Loading BokehJS ...

# Check other pathways the genes (+/- and sig in mult. cancers) are part of

In [21]:
#gnm = list(genes_pn_mult.Proteomics.unique())
#print(gnm)
enr3 = gp.enrichr(gene_list = genes_mult, description='Tumor_partition', gene_sets='Reactome_2016', 
                       outdir='/Enrichr')

In [22]:
enr3.res2d.head()

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,"Cell Cycle, Mitotic Homo sapiens R-HSA-69278",38/462,1.408343e-63,2.154764e-60,0,0,43.290043,6264.954425,TOP2A;AHCTF1;PCNA;MCM7;DCTN1;BUB1B;PPP2R2A;ORC...,Reactome_2016
1,Cell Cycle Homo sapiens R-HSA-1640170,38/566,4.241800e-60,3.244977e-57,0,0,35.335689,4830.746211,TOP2A;AHCTF1;PCNA;MCM7;DCTN1;BUB1B;PPP2R2A;ORC...,Reactome_2016
2,M Phase Homo sapiens R-HSA-68886,18/268,2.881092e-24,1.469357e-21,0,0,35.349568,1916.083489,RANBP2;AHCTF1;NUP210;NUP155;NUP133;PRKCB;BUB1B...,Reactome_2016
3,DNA strand elongation Homo sapiens R-HSA-69190,11/32,2.952657e-23,1.129391e-20,0,0,180.921053,9385.596522,GINS2;RFC3;PCNA;RFC4;MCM7;RFC2;GINS4;MCM3;MCM4...,Reactome_2016
4,Synthesis of DNA Homo sapiens R-HSA-69239,12/97,2.042530e-19,6.250142e-17,0,0,65.111232,2802.057139,ORC5;GINS2;RFC3;RFC4;PCNA;MCM7;RFC2;GINS4;MCM3...,Reactome_2016


In [23]:
j = 2
print(enr3.res2d.Term[j])
enr3.res2d.Genes[j]

M Phase Homo sapiens R-HSA-68886


'RANBP2;AHCTF1;NUP210;NUP155;NUP133;PRKCB;BUB1B;NUP153;PPP2R2A;NDC80;CENPF;XPO1;NUP85;PPP2R2D;TPR;NUF2;RAE1;MAD2L1'

In [24]:
# Look at p-values 
certain_gene = 'CHEK1'
all_df = pd.read_csv(root+R'\all_heatmap.csv')
gene_df = all_df.loc[all_df['Proteomics'] == certain_gene]
gene_df

,Proteomics,P_Value,Medians,Cancer
915,CHEK1,0.014741,0.461686,Gbm
11802,CHEK1,0.066440,0.312344,Hnscc
29569,CHEK1,0.977477,0.010050,Luad
35608,CHEK1,0.999914,0.231300,Lscc
41882,CHEK1,0.597249,0.536650,Brca
53446,CHEK1,0.985666,0.078545,Ov
60481,CHEK1,0.128131,-0.528000,En
